In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from sklearn import preprocessing

In [9]:
data_a_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_A.csv", low_memory = False);
data_b_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_B.csv", low_memory = False);
data_c_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_C.csv", low_memory = False);
data_d_2014 = pd.read_csv("../data/2014_loan_data/LoanStats_2014_D.csv", low_memory = False);

data_a_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_A.csv", low_memory = False);
data_b_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_B.csv", low_memory = False);
data_c_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_C.csv", low_memory = False);
data_d_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_D.csv", low_memory = False);
data_e_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_E.csv", low_memory = False);
data_f_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_F.csv", low_memory = False);
data_g_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_G.csv", low_memory = False);
data_h_2015 = pd.read_csv("../data/2015_loan_data/LoanStats_2015_H.csv", low_memory = False);

In [10]:
data_2014 = pd.concat([data_a_2014, data_b_2014, data_c_2014, data_d_2014], ignore_index=True)

data_2015 = pd.concat([data_a_2015, data_b_2015, data_c_2015, data_d_2015, 
                       data_e_2015, data_f_2015, data_g_2015, data_h_2015], 
                       ignore_index=True)

In [11]:
def format_revol(val):
    percent = val[:-1]
    return float(percent)/100

def format_interest(val):
    percent = val[:-1]
    return float(percent)/100

def format_term(val):
    term = val[1:3]
    return float(term)/12

In [19]:
def initial_model(df, columns, columns_to_encode, columns_to_normalize):
    min_max_scaler = preprocessing.MinMaxScaler()
    new_df = df.copy()
    new_df = new_df[columns]
    new_df = pd.get_dummies(new_df, columns = columns_to_encode)
    new_df['amnt'] = new_df['funded_amnt']
    new_df[columns_to_normalize] = min_max_scaler.fit_transform(new_df[columns_to_normalize])
    return new_df

def clean_data(old_df):
    df = old_df.copy()
    columns_to_drop = ['id', 'member_id', 'emp_title']
    df = df.drop(columns_to_drop, axis = 1)
    
    cols = ['funded_amnt', 'emp_length', 'home_ownership', 'int_rate', 'purpose', 'total_pymnt',
        'annual_inc', 'verification_status', 'dti', 'loan_status', 'revol_util', 'grade', 'term']
    cols_encode = ['emp_length', 'home_ownership', 'verification_status', 'grade', 'purpose']
    cols_normalize = ['funded_amnt', 'annual_inc', 'dti']
    
    df = initial_model(df, cols, cols_encode, cols_normalize)
    
    df['term'] = df['term'].map(format_term, na_action = 'ignore')
    df['int_rate'] = df['int_rate'].map(format_interest, na_action='ignore')
    df['revol_util'] = df['revol_util'].map(format_revol, na_action='ignore')
    
    df[df['revol_util'].isnull()] = np.mean(df['revol_util'])
    df[df['int_rate'].isnull()] = np.mean(df['int_rate'])
    df = df[df.loan_status != 'Current']
    
    df['paid'] = 1;
    df.loc[df.loan_status == 'Fully Paid', 'paid'] = 0;
    
    return df

In [20]:
df_2014 = clean_data(data_2014)

/Users/hugoramambason/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [23]:
df_2015 = clean_data(data_2015)

/Users/hugoramambason/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [15]:
df_2015.head()

,funded_amnt,int_rate,total_pymnt,annual_inc,dti,loan_status,revol_util,term,emp_length_1 year,emp_length_10+ years,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,amnt,paid
1,0.264706,0.1288,11316.890190,0.009620,0.009449,Fully Paid,0.417,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000.0,0
2,0.500000,0.0749,19771.097253,0.007895,0.025676,Fully Paid,0.332,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18000.0,0
3,0.558824,0.0532,21631.724329,0.006842,0.018849,Fully Paid,0.030,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20000.0,0
6,0.484559,0.1399,13449.650000,0.004184,0.023604,Charged Off,0.523,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17475.0,1
7,0.588235,0.1344,24473.227622,0.005753,0.012152,Fully Paid,0.268,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21000.0,0


In [16]:
df_2014.to_csv('cleaned_2014', index = False)
df_2015.to_csv('cleaned_2015', index = False)